# How to Train a Florence 2 Model Using the Florence2 class.
---



## Pre-work

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

## Install needed packages

In [ ]:
# Install necessary packages
!pip install torch transformers peft pillow matplotlib seaborn tqdm ultralytics

### Mount your Google Drive
---
You can comment this out if working on the ephemeral session but it is recommended.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ask the user for the working directory name
import os
working_dir = input("Enter the name for the working directory: ")
working_path = os.path.join('/content/drive/MyDrive', working_dir)
os.makedirs(working_path, exist_ok=True)

# Change directory to the working directory
%cd {working_path}

## Clone BaseballCV Repo and set as Current Directory

In [ ]:
# Clone the repository
!git clone https://github.com/dylandru/BaseballCV.git


### Install requirements for BaseballCV

In [ ]:
# Change directory to the repository
%cd BaseballCV
!pip install -r requirements.txt

### Import scripts for dataset and model handling

In [ ]:
# Import necessary modules
import sys
from scripts import LoadTools
from scripts import Florence2

### Download the Baseball Only Dataset
---

You can change the dataset name and download it, depending on your needs.

In [ ]:
# Initialize the LoadTools class
load_tools = LoadTools()

# Download the 'baseball' dataset
dataset_path = load_tools.load_dataset('baseball')
print(f"Dataset downloaded to: {os.path.abspath(dataset_path)}")

In [ ]:
# Define the classes for the dataset
classes = {
    0: 'baseball',
    1: 'glove',
    2: 'bat',
    3: 'player'
}

### Let's initialize the model class, Florence2 in this example.

In [ ]:
# Initialize the Florence2 class
florence_model = Florence2()

### Finally, let's train the model.

In [ ]:
# Fine-tune the model

# First, we have a basic template for default parameters:

metrics = florence_model.finetune(
    dataset=dataset_path,
    classes=classes,
    train_test_split=(80, 10, 10),
    epochs=2,
    save_dir="model_checkpoints",
    num_workers=0,
    )

# You can customize it more, as per this other example:


"""
metrics = florence_model.finetune(
    dataset=dataset_path,
    classes=classes,
    train_test_split=(80, 10, 10),
    epochs=20,
    lr=4e-6,
    save_dir="model_checkpoints",
    num_workers=0,
    lora_r=8,
    lora_scaling=8,
    patience=5,
    lora_dropout=0.05,
    warmup_epochs=1,
    lr_schedule="cosine"
)
"""



### Perform Inference on a Sample Image

In [ ]:
sample_image_path = os.path.join(dataset_path, 'images', 'sample_image.jpg')  # Replace with an actual image path
results = florence_model.inference(image_path=sample_image_path, task="<OD>")

# Print the results
print(results)

### If you want to load the resulting model later, you can use the following code:
---
Replace {working_path} accordingly.

In [ ]:
# Define the path to the saved model checkpoint
saved_model_path = os.path.join(working_path, 'model_checkpoints', 'best_model')

# Initialize the Florence2 class with the saved model path
florence_model = Florence2(model_id=saved_model_path)

# Verify the model is loaded correctly
print(f"Model loaded from: {saved_model_path}")
